# M5 EDA in Julia

## Preparation

In [1]:
using DataFrames
import CSV
import Parquet
using CategoricalArrays
using FilePaths
using FilePathsBase: /
using Chain

See note: https://dataframes.juliadata.org/stable/man/getting_started/#Installation

In [2]:
ENV["LINES"] = 10
ENV["COLUMNS"] = 1000;

In [3]:
path_data = parent(parent(cwd())) / "data";

In [50]:
mkpath(path_data / "clean")
macro xpath(arg)
    return path_data / "clean" / (string(arg) * ".csv")
end
macro apath(arg)
    return string(path_data / "clean" / (string(arg) * ".arrow"))
end

@apath (macro with 1 method)

## Load and preview files

### Calendar

In [4]:
raw_calendar = CSV.read(path_data / "calendar.csv", DataFrame)

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
,Date…,Int64,String,Int64,Int64,Int64,String,String?,String?,String?,String?,Int64,Int64,Int64
1,2011-01-29,11101,Saturday,1,1,2011,d_1,missing,missing,missing,missing,0,0,0
2,2011-01-30,11101,Sunday,2,1,2011,d_2,missing,missing,missing,missing,0,0,0
3,2011-01-31,11101,Monday,3,1,2011,d_3,missing,missing,missing,missing,0,0,0
4,2011-02-01,11101,Tuesday,4,2,2011,d_4,missing,missing,missing,missing,1,1,0
5,2011-02-02,11101,Wednesday,5,2,2011,d_5,missing,missing,missing,missing,1,0,1
6,2011-02-03,11101,Thursday,6,2,2011,d_6,missing,missing,missing,missing,1,1,1
7,2011-02-04,11101,Friday,7,2,2011,d_7,missing,missing,missing,missing,1,0,0
8,2011-02-05,11102,Saturday,1,2,2011,d_8,missing,missing,missing,missing,1,1,1
9,2011-02-06,11102,Sunday,2,2,2011,d_9,SuperBowl,Sporting,missing,missing,1,1,1


In [5]:
snap = @chain raw_calendar begin
    stack(r"snap", :date, variable_name=:state_raw, value_name=:snap)
    select(:date, :state_raw => ByRow(x -> String(chop(x, head=5, tail=0))) => :state_id, :snap)
end

,date,state_id,snap
,Date…,String,Int64
1,2011-01-29,CA,0
2,2011-01-30,CA,0
3,2011-01-31,CA,0
4,2011-02-01,CA,1
5,2011-02-02,CA,1
6,2011-02-03,CA,1
7,2011-02-04,CA,1
8,2011-02-05,CA,1
9,2011-02-06,CA,1


In [6]:
cal_events = select(
    raw_calendar, 
    :date, 
    (x => (y -> categorical(coalesce.(y, "(No Event)"))) => x 
        for x ∈ [:event_name_1, :event_type_1, :event_name_2, :event_type_2])...
)

,date,event_name_1,event_type_1,event_name_2,event_type_2
,Date…,Cat…,Cat…,Cat…,Cat…
1,2011-01-29,(No Event),(No Event),(No Event),(No Event)
2,2011-01-30,(No Event),(No Event),(No Event),(No Event)
3,2011-01-31,(No Event),(No Event),(No Event),(No Event)
4,2011-02-01,(No Event),(No Event),(No Event),(No Event)
5,2011-02-02,(No Event),(No Event),(No Event),(No Event)
6,2011-02-03,(No Event),(No Event),(No Event),(No Event)
7,2011-02-04,(No Event),(No Event),(No Event),(No Event)
8,2011-02-05,(No Event),(No Event),(No Event),(No Event)
9,2011-02-06,SuperBowl,Sporting,(No Event),(No Event)


In [7]:
cal_info = select(raw_calendar, :date, :year, :month, :weekday)

,date,year,month,weekday
,Date…,Int64,Int64,String
1,2011-01-29,2011,1,Saturday
2,2011-01-30,2011,1,Sunday
3,2011-01-31,2011,1,Monday
4,2011-02-01,2011,2,Tuesday
5,2011-02-02,2011,2,Wednesday
6,2011-02-03,2011,2,Thursday
7,2011-02-04,2011,2,Friday
8,2011-02-05,2011,2,Saturday
9,2011-02-06,2011,2,Sunday


In [8]:
cal_mapping = select(raw_calendar, :date, :d, :wm_yr_wk)

,date,d,wm_yr_wk
,Date…,String,Int64
1,2011-01-29,d_1,11101
2,2011-01-30,d_2,11101
3,2011-01-31,d_3,11101
4,2011-02-01,d_4,11101
5,2011-02-02,d_5,11101
6,2011-02-03,d_6,11101
7,2011-02-04,d_7,11101
8,2011-02-05,d_8,11102
9,2011-02-06,d_9,11102


### Sales data

In [9]:
raw_sales = CSV.read(path_data / "sales_train_validation.csv", DataFrame)

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,d_32,d_33,d_34,d_35,d_36,d_37,d_38,d_39,d_40,d_41,d_42,d_43,d_44,d_45,d_46,d_47,d_48,d_49,d_50,d_51,d_52,d_53,d_54,d_55,d_56,d_57,d_58,d_59,d_60,d_61,d_62,d_63,d_64,d_65,d_66,d_67,d_68,d_69,d_70,d_71,d_72,d_73,d_74,d_75,d_76,d_77,d_78,d_79,d_80,d_81,d_82,d_83,d_84,d_85,d_86,d_87,d_88,d_89,d_90,d_91,d_92,d_93,d_94,d_95,d_96,d_97,d_98,d_99,d_100,d_101,d_102,d_103,d_104,d_105,d_106,d_107,d_108,d_109,d_110,d_111,d_112,d_113,d_114,d_115,d_116,d_117,d_118,d_119,d_120,d_121,d_122,d_123,d_124,d_125,d_126,d_127,d_128,d_129,d_130,d_131
,String,String,String,String,String,String,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,2,0,1,0,0,0,0,0,0,2,0,1,0,0,1,1,1,0,2,3,1,0,0,0,0,0,1,0,0,1,1,0,0,1,0,0,1,2,3,0,2,0,0,2,2,0,0,2,1,2,1,1,1,2,0,0,1,0,1,3,1,0,0,0,0,3,5,2,2,1,1,1,1,1,1,0,0,2,1,1,1,2,0,0,0,2,5,6,0,0,0
5,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,6,0,3,2,3,5,3,1,0,0,1,0,2,2,4,0,0,3
6,HOBBIES_1_006_CA_1_validation,HOBBIES_1_006,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,HOBBIES_1_007_CA_1_validation,HOBBIES_1_007,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,HOBBIES_1_008_CA_1_validation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,12,15,0,0,0,4,6,5,7,0,13,2,2,20,9,3,0,

Get hierarchies

In [10]:
hier_product = select(raw_sales, [:item_id, :dept_id, :cat_id]) |> unique

,item_id,dept_id,cat_id
,String,String,String
1,HOBBIES_1_001,HOBBIES_1,HOBBIES
2,HOBBIES_1_002,HOBBIES_1,HOBBIES
3,HOBBIES_1_003,HOBBIES_1,HOBBIES
4,HOBBIES_1_004,HOBBIES_1,HOBBIES
5,HOBBIES_1_005,HOBBIES_1,HOBBIES
6,HOBBIES_1_006,HOBBIES_1,HOBBIES
7,HOBBIES_1_007,HOBBIES_1,HOBBIES
8,HOBBIES_1_008,HOBBIES_1,HOBBIES
9,HOBBIES_1_009,HOBBIES_1,HOBBIES


In [11]:
hier_location = select(raw_sales, [:store_id, :state_id]) |> unique

,store_id,state_id
,String,String
1,CA_1,CA
2,CA_2,CA
3,CA_3,CA
4,CA_4,CA
5,TX_1,TX
6,TX_2,TX
7,TX_3,TX
8,WI_1,WI
9,WI_2,WI


We need to reshape this from wide to long format, which is pretty easy:

In [12]:
# unst_sales = stack(raw_sales, r"d_", [:item_id, :store_id], variable_name=:d, value_name=:sales)
# sales = leftjoin(unst_sales, select(calendar, [:date, :d]), on=:d)
# disallowmissing!(sales, :date)  # we know all dates must be available
# select!(sales, Not(:d))

Note that the following would do this in a single expression (we can omit the first arg, or use `_,` instead)

In [13]:
sales = @chain raw_sales begin
    stack(r"d_", [:item_id, :store_id], variable_name=:d, value_name=:sales)
    leftjoin(select(cal_mapping, [:date, :d]), on=:d)
    disallowmissing(:date)
    select(Not(:d))
end

,item_id,store_id,sales,date
,String,String,Int64,Date…
1,HOBBIES_1_001,CA_1,0,2011-01-29
2,HOBBIES_1_002,CA_1,0,2011-01-29
3,HOBBIES_1_003,CA_1,0,2011-01-29
4,HOBBIES_1_004,CA_1,0,2011-01-29
5,HOBBIES_1_005,CA_1,0,2011-01-29
6,HOBBIES_1_006,CA_1,0,2011-01-29
7,HOBBIES_1_007,CA_1,0,2011-01-29
8,HOBBIES_1_008,CA_1,12,2011-01-29
9,HOBBIES_1_009,CA_1,2,2011-01-29


### Selling Prices

In [14]:
raw_sell_prices = CSV.read(path_data / "sell_prices.csv", DataFrame)

,store_id,item_id,wm_yr_wk,sell_price
,String,String,Int64,Float64
1,CA_1,HOBBIES_1_001,11325,9.58
2,CA_1,HOBBIES_1_001,11326,9.58
3,CA_1,HOBBIES_1_001,11327,8.26
4,CA_1,HOBBIES_1_001,11328,8.26
5,CA_1,HOBBIES_1_001,11329,8.26
6,CA_1,HOBBIES_1_001,11330,8.26
7,CA_1,HOBBIES_1_001,11331,8.26
8,CA_1,HOBBIES_1_001,11332,8.26
9,CA_1,HOBBIES_1_001,11333,8.26


In [16]:
sell_prices = @chain raw_sell_prices begin
    leftjoin(select(cal_mapping, [:date, :wm_yr_wk]), on=:wm_yr_wk)  # 7x larger
    disallowmissing(:date)
    select(Not(:wm_yr_wk))
end

,store_id,item_id,sell_price,date
,String,String,Float64,Date…
1,CA_1,HOBBIES_1_001,9.58,2013-07-13
2,CA_1,HOBBIES_1_001,9.58,2013-07-14
3,CA_1,HOBBIES_1_001,9.58,2013-07-15
4,CA_1,HOBBIES_1_001,9.58,2013-07-16
5,CA_1,HOBBIES_1_001,9.58,2013-07-17
6,CA_1,HOBBIES_1_001,9.58,2013-07-18
7,CA_1,HOBBIES_1_001,9.58,2013-07-19
8,CA_1,HOBBIES_1_001,9.58,2013-07-20
9,CA_1,HOBBIES_1_001,9.58,2013-07-21


## Bringing it together

In [15]:
# snap, cal_events, cal_info, cal_mapping, hier_product, hier_location, sales, sell_prices

**FIXME**: Change to Parquet or Arrow files.

In [52]:
CSV.write(@xpath(snap), snap)
CSV.write(@xpath(cal_events), cal_events)
CSV.write(@xpath(cal_info), cal_info)
CSV.write(@xpath(cal_mapping), cal_mapping)
CSV.write(@xpath(hier_product), hier_product)
CSV.write(@xpath(hier_location), hier_location)
CSV.write(@xpath(sales), sales)
CSV.write(@xpath(sell_prices), sell_prices)
"done"

"done"